In [ ]:
import requests
import folium
import geopandas as gpd
import pandas as pd

In [ ]:
geojson_jakarta_path = r"C:/GD_2025/6_spatial_data_science/1_location_fnb/2_pipeline/location_intelligence_potential_sales_areas/data/vektor/jakarta.geojson"
gdf = gpd.read_file(geojson_jakarta_path)
minx, miny, maxx, maxy = gdf.total_bounds 

print(f"bounding box Jakarta: ({miny}, {minx}, {maxy}, {maxx})")

In [ ]:
query = f"""
[out:json];
(
  node["tourism"="hotel"]({miny},{minx},{maxy},{maxx});
  node["building"="office"]({miny},{minx},{maxy},{maxx});
  node["office"]({miny},{minx},{maxy},{maxx});
  node["amenity"="business_centre"]({miny},{minx},{maxy},{maxx});
  node["building"="commercial"]({miny},{minx},{maxy},{maxx});
  node["building"="apartments"]({miny},{minx},{maxy},{maxx});
);
out center;
"""

overpass_url = "http://overpass-api.de/api/interpreter"
response = requests.get(overpass_url, params={'data': query})
data = response.json()

In [ ]:
places = []

for element in data['elements']:
    name = element.get('tags', {}).get('name', 'undefined')
    tags = element.get("tags", {})

    if "tourism" in tags and tags["tourism"] == "hotel":
        category = "Hotel"
    elif "building" in tags and tags["building"] in ["office", "commercial"]:
        category = "Gedung Perkantoran"
    elif "office" in tags:  
        category = "Gedung Perkantoran"
    elif "amenity" in tags and tags["amenity"] == "business_centre":
        category = "Pusat Bisnis"
    else:
        category = "Apartemen"
        
    lat = element.get('lat', None)
    lon = element.get('lon', None)
    places.append([name, category, lat, lon])

df_places = pd.DataFrame(places, columns=["nama", "kategori", "latitude", "longitude"])
print(df_places)

In [ ]:
latitude = -6.195242532131491
longitude = 106.8230052861102

m = folium.Map(location=[latitude, longitude], zoom_start=11)

folium.GeoJson(
    geojson_jakarta_path,
    name="Jakarta Area",
    style_function=lambda feature: {
        "fillColor": "#FF7517",  
        "color": "#FF7517",  
        "weight": 1,  
        "fillOpacity": 0.5  
    }
).add_to(m)

for _, row in df_places.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"{row['nama']} ({row['kategori']})",
        icon=folium.Icon(color="blue" if row["kategori"] == "Hotel" else "red" if row["kategori"] == "Gedung Perkantoran" else "green")
    ).add_to(m)

folium.LayerControl().add_to(m)

m


In [ ]:
from shapely.geometry import Point

jakarta_areas = gpd.read_file(geojson_jakarta_path)

customer_location = [Point(xy) for xy in zip(df_places["longitude"], df_places["latitude"])]
gdf_location = gpd.GeoDataFrame(df_places, geometry=customer_location, crs='EPSG:4326')

result_customer_location = gdf_location.clip(jakarta_areas)

m = folium.Map(location=[latitude, longitude], zoom_start=11)

folium.GeoJson(
    geojson_jakarta_path,
    name="Jakarta Area",
    style_function=lambda feature: {
        "fillColor": "#FF7517",  
        "color": "#FF7517",  
        "weight": 1,  
        "fillOpacity": 0.5  
    }
).add_to(m)

for _, row in result_customer_location.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"{row['nama']} ({row['kategori']})",
        icon=folium.Icon(color="blue" if row["kategori"] == "Hotel" else "red" if row["kategori"] == "Gedung Perkantoran" else "green")
    ).add_to(m)

folium.LayerControl().add_to(m)

m